In [1]:
import pandas as pd
import numpy as np
from Funcoes_auxiliares.func_aux import *

In [3]:
pasta_origem = r"D:/ETLSINASC"
year =[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [4]:
def function_list_path_files(path: str):
    """
    Maps files to a specified path
    :param path (str): Path of the folder that should be mapped
    :return: Returns a list with the path of all files in the folder.
    """
    arquivos = []
    for diretorio_raiz, _, arquivos_na_pasta in os.walk(path):
        for arquivo in arquivos_na_pasta:
            endereco_completo = os.path.join(diretorio_raiz, arquivo)
            arquivos.append(endereco_completo)
    return arquivos

In [6]:
lista_arq = function_list_path_files(path=pasta_origem)

In [8]:
# Filter addresses according to the years of interest
enderecos_filtrados = [endereco for endereco in lista_arq if any(str(ano) in endereco for ano in year)]

In [10]:
variaveis = [
    'data_nasc'
    , 'ano_nasc'
    , 'nasc_MUNNOMEX' #'ocor_MUNNOMEX'
    , 'res_MUNNOMEX'
    , 'nasc_CAPITAL'
    , 'res_CAPITAL'
    , 'nasc_REGIAO'
    , 'res_REGIAO'
    , 'nasc_SIGLA_UF'
    , 'res_SIGLA_UF'
    , 'IDADEMAE'
    , 'ESCMAE2010'
    , 'GRAVIDEZ'
    , 'SEMAGESTAC'
    , 'SEXO'
    , 'def_sexo'
    , 'PESO'

    ]

In [12]:
df = [pd.read_csv(i)[variaveis] for i in enderecos_filtrados]

<ipython-input-12-03f65b3af858>:1: DtypeWarning: Columns (30,112,113,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  df = [pd.read_csv(i)[variaveis] for i in enderecos_filtrados]
<ipython-input-12-03f65b3af858>:1: DtypeWarning: Columns (25,114,115,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = [pd.read_csv(i)[variaveis] for i in enderecos_filtrados]
<ipython-input-12-03f65b3af858>:1: DtypeWarning: Columns (117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = [pd.read_csv(i)[variaveis] for i in enderecos_filtrados]
<ipython-input-12-03f65b3af858>:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = [pd.read_csv(i)[variaveis] for i in enderecos_filtrados]
<ipython-input-12-03f65b3af858>:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = [pd.read_csv(i)[variavei

In [13]:
df_sinasc = pd.concat(df)

In [14]:
del df

In [15]:
df_sinasc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28445535 entries, 0 to 22548
Data columns (total 17 columns):
 #   Column         Dtype  
---  ------         -----  
 0   data_nasc      object 
 1   ano_nasc       int64  
 2   nasc_MUNNOMEX  object 
 3   res_MUNNOMEX   object 
 4   nasc_CAPITAL   object 
 5   res_CAPITAL    object 
 6   nasc_REGIAO    object 
 7   res_REGIAO     object 
 8   nasc_SIGLA_UF  object 
 9   res_SIGLA_UF   object 
 10  IDADEMAE       float64
 11  ESCMAE2010     float64
 12  GRAVIDEZ       int64  
 13  SEMAGESTAC     float64
 14  SEXO           int64  
 15  def_sexo       object 
 16  PESO           float64
dtypes: float64(4), int64(3), object(10)
memory usage: 3.8+ GB


In [16]:
df_sinasc['FLAG_BASE'] = 'SINASC'
df_sinasc['idademae_faixa'] = [func_categorize_idademae(i) for i in df_sinasc['IDADEMAE']]
df_sinasc['escolaridade_mae'] = [func_categorize_escolmae(i) for i in df_sinasc['ESCMAE2010']]
df_sinasc['tipo_gravidez'] = [func_categorize_gravidez(i) for i in df_sinasc['GRAVIDEZ']]
df_sinasc['idade_gestacao_faixa'] = [func_categorize_idade_gest(i) for i in df_sinasc['SEMAGESTAC']]
df_sinasc['peso_faixa'] = [func_categorize_peso(i) for i in df_sinasc['PESO']]

df_sinasc = df_sinasc[
    [
    'data_nasc'
    , 'ano_nasc'
    , 'nasc_MUNNOMEX'
    , 'res_MUNNOMEX'
    , 'nasc_CAPITAL'
    , 'res_CAPITAL'
    , 'nasc_REGIAO'
    , 'res_REGIAO'
    , 'nasc_SIGLA_UF'
    , 'res_SIGLA_UF'
    , 'IDADEMAE'
    , 'idademae_faixa'
    , 'ESCMAE2010'
    , 'escolaridade_mae'
    , 'GRAVIDEZ'
    , 'tipo_gravidez'
    , 'SEMAGESTAC'
    , 'idade_gestacao_faixa'
    , 'SEXO'
    , 'def_sexo'
    , 'PESO'
    , 'peso_faixa'
    , 'FLAG_BASE'
    ]
]

df_sinasc.to_csv('base_suja/base_sinasc_suja.csv', index=False)

# Tratando a base

In [ ]:
# Configuration to show more rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:
df_sinasc.info()

In [ ]:
df_sinasc.head()

In [ ]:
# Duplicate data 
duplicados = df_sinasc.duplicated()
soma = duplicados.sum()
f"""The dataset has {soma} duplicate rows, which represents {round((soma/len(df_sinasc)) * 100, 2)} %"""

In [ ]:
# Frequency of duplicates by res_SIGLA_UF and ano_evento
df_duplicados = df_sim_dofet[duplicados]
df_duplicados.value_counts(['ano_nasc', 'res_SIGLA_UF'])

In [ ]:
# Counting missing values
df_sinasc_ = df_sinasc[df_sinasc['FLAG_BASE']=='SIM_DOFET']
missing_count_sim_dofet = df_sim.isnull().sum()  # counts the null values in each column
missing_percent_sim_dofet = round((missing_count_sim_dofet / len(df_sim)) * 100,2)  # calculates the percentage of null values
missing_data_sim_dofet = pd.DataFrame({'Missing Count': missing_count_sim_dofet, 'Missing Percentage': missing_percent_sim_dofet})
missing_data_sim_dofet.index.name = 'Variable'  # sets the index name to 'Variable'
missing_data_sim_dofet.reset_index(inplace=True)  # resets the index to make 'Variable' a column
missing_data_sim_dofet['BASE'] = 'SIM_DOFET'


# Sorting the DataFrame by the highest missing frequencies
missing_data_sorted = missing_data_sim_dofet.sort_values(by=['BASE', 'Missing Count'], ascending=False)

missing_data_sorted[missing_data_sorted['Missing Percentage'] > 0]